## Feature Extraction
This file serves as a prototyping and exploration of feature extraction fromt he scraped Google job listings data. 
Features include: 
- Salary

In [1]:
import pandas as pd
from transformers import pipeline

### Read in Scraped Data

In [52]:
df = pd.read_csv('../data.csv')
df.head()

,title,company_name,location,via,description,job_highlights,detected_extensions,job_id
0,Junior Data Scientist,ING,Amsterdam,ING Careers,As the data driven mindset is more and more em...,['As the data driven mindset is more and more ...,"{'posted_at': '6 days ago', 'schedule_type': '...",eyJqb2JfdGl0bGUiOiJKdW5pb3IgRGF0YSBTY2llbnRpc3...
1,"JUNIOR DATA SCIENTIST - Dubai, UAE",Cobblestone Energy,Utrecht,LinkedIn,"Location: Dubai, UAE (We provide visa sponsors...","[""Location: Dubai, UAE (We provide visa sponso...","{'posted_at': '4 hours ago', 'schedule_type': ...",eyJqb2JfdGl0bGUiOiJKVU5JT1IgREFUQSBTQ0lFTlRJU1...
2,Data Scientist Mobiliteit,TNO,The Hague,TNO,Halen we in Nederland de klimaatdoelen op het ...,['Halen we in Nederland de klimaatdoelen op he...,"{'posted_at': '5 days ago', 'schedule_type': '...",eyJqb2JfdGl0bGUiOiJEYXRhIFNjaWVudGlzdCBNb2JpbG...
3,Data Scientist Real Estate for Catella Investm...,Catella Investment Management Benelux,Maastricht,Limburgvac,As a Data Scientist in the Research & Investme...,['As a Data Scientist in the Research & Invest...,"{'posted_at': '20 hours ago', 'schedule_type':...",eyJqb2JfdGl0bGUiOiJEYXRhIFNjaWVudGlzdCBSZWFsIE...
4,Data Scientist,Effectory,Amsterdam,Effectory Jobs,Improving the working lives of millions of peo...,['Improving the working lives of millions of p...,{'schedule_type': 'Full–time'},eyJqb2JfdGl0bGUiOiJEYXRhIFNjaWVudGlzdCIsImh0aW...


### Salary Extraction
There exist three possibilities of salary formatting within the job listings. 
1. The job listing will contain no salary information.
2. The job listing will contain a salary range. 
3. The job listing will contain a specified salary value

The accumulated salaries are then required to be adjusted into a single currency for comparison.

In [53]:
## JOB Highlights cleaning ## 
df['job_highlights'] = df['job_highlights'].replace(r'\n\n|\n•|\n|\\n|\\n•|•', '', regex=True)  # Remove new line char
df['job_highlights'] = df['job_highlights'].str.slice(2, -2)  # Remove [] and additional "" marks

In [ ]:
## NLP Approach using BERT Question and Answer model ## 
qa_model = pipeline("question-answering", 
                    model='distilbert-base-cased-distilled-squad') # Define the model 

question = "What is the salary or salary range for the job?"  # Define the quesiton to be answered
highlights = df['job_highlights'].head(20).to_list()  # Convert highlights to list of str

# Extract salary from highlights and place in salary column
df['salary'] = df['job_highlights'].map(lambda x: "NaN" if qa_model(question=question, context=x)['score'] < 0.3
                                        else qa_model(question=question, context=x)['answer'])
print(df.salary[0:50])